<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Label-the-Rodent's-Orientations-Within-Frame-Ranges" data-toc-modified-id="Label-the-Rodent's-Orientations-Within-Frame-Ranges-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Label the Rodent's Orientations Within Frame Ranges</a></span></li><li><span><a href="#Prepare-Train-Validation-Datasets" data-toc-modified-id="Prepare-Train-Validation-Datasets-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare Train-Validation Datasets</a></span></li><li><span><a href="#Fit-or-Evaluate-the-Flip-Classifier-Model" data-toc-modified-id="Fit-or-Evaluate-the-Flip-Classifier-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fit or Evaluate the Flip Classifier Model</a></span></li><li><span><a href="#Correct-Extracted-Dataset-Using-Train-Flip-Classifier-Model" data-toc-modified-id="Correct-Extracted-Dataset-Using-Train-Flip-Classifier-Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Correct Extracted Dataset Using Train Flip Classifier Model</a></span><ul class="toc-item"><li><span><a href="#Apply-a-flip-classifier-to-correct-the-extracted-dataset" data-toc-modified-id="Apply-a-flip-classifier-to-correct-the-extracted-dataset-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Apply a flip classifier to correct the extracted dataset</a></span></li><li><span><a href="#Preview-Corrected-Sessions" data-toc-modified-id="Preview-Corrected-Sessions-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Preview Corrected Sessions</a></span></li></ul></li></ul></div>

Flip classifiers are RandomForestClassifier models that MoSeq2-Extract uses to ensure that the mouse is always extracted with the mouse's nose pointing to the right and tail to the left. This notebook is a streamlined utility and guide for preparing data and training a model that handles your specific data acquisition use case.

To use this notebook, you must first extract some data using MoSeq2-Extract to use as training data for the flip classifier model. 100K frames are optimal for training the flip classifier. 

This can be an iterative process if your data contains large amounts of flips throughout the extractions. On your first iteration, it is acceptable to extract the data without a flip-classifier. After training a new flip classifier, you may apply it to your dataset to correct the flips without having to re-extract the data before going into the PCA step.

<center><img src="https://drive.google.com/uc?export=view&id=1cOwyen2Siy-_wJ1HcE0PmMUi3Lcgcwwa"></center>

## Label the Rodent's Orientations Within Frame Ranges
Use this interactive tool to build your training dataset for the flip classifier model. Select a range of frames and identify whether the rodent is facing left or facing right. The ranges of frames are used to build your training set.

**Instructions**
- **Specify the config file** in the `config_path` field. This will give the model the correct cleaning parameters for your data. MAKE SURE THESE ARE THE SAME AS THE ONES YOU PLAN TO USE IN EXTRACTION.
- **Specify the data folder** in the `input_dir` field. The GUI works best if it is the folder specified while running the `aggregate results` step in extraction. 
- **Specify the path for the resulting model** in the `model_path` field. For example, `./flip-classifier-azure-ephys.pkl`.
- **Run the following cell** to set the parameters and initialize the Data Labeller.

In [1]:
from moseq2_app.flip.widget import FlipClassifierWidget
from moseq2_app.flip.train import create_training_dataset, train_classifier, save_classifier, CleanParameters
from moseq2_app.util import read_yaml
import panel as pn
pn.extension()

# specify paths
config_path = '/path/to/config.yaml'
input_dir = '/path/to/aggregate_results' # Specify the data folder
model_path = '/path/to/flip/new-flip-tst.pkl' ## e.g. ./flip-classifier-azure-ephys.pkl

FF = FlipClassifierWidget(input_dir)

/home/jal5475/.miniconda/envs/moseq2-dev/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


**Instructions:**
- **Run the following cell** to launch the Data Labeller GUI.
- **Select the target session from the dropdown menu** and start labeling.
- **Drag the slider** to select a frame index to preview.
- **Click `Start Range`** to starting selecting the range. **Click `Forward`** until the mouse faces the different direction. **Click `Facing Left` or `Facing Right`** to specify the correct orientation for the range of frames. After specifying the orientation, the selected frames will be added to the dataset used to train the model.
- **Click `Cancel Select`** to cancel the selection.

<!-- **Note**: The `Current Total Selected` section turns green when there are enough labeled frames to train the model. If your frame selection was interrupted for any reason, and you would like to relaunch the tool with all of your previously selected frame ranges, uncomment the code in the following cell and run the cell.

If two frame ranges are selected with overlapping frames, the training set will only include the unique selected indices, removing duplicates. 
 -->

In [2]:
FF.show()

Row
    [0] Column(width=325)
        [0] Select(name='Session', options=['2019-03-20_14-37-10_amph...], value='2019-03-20_14-37-10_amphe...)
        [1] IntSlider(end=35973, name='Current Frame')
        [2] Button(button_type='primary', name='Start Range')
        [3] Row
            [0] Button(button_type='primary', name='Backward', width=142)
            [1] Button(button_type='primary', name='Forward', width=142)
        [4] Markdown(str, visible=False)
        [5] Row
            [0] Button(button_type='success', name='Facing Left', visible=False, width=140)
            [1] Button(button_type='success', name='Facing Right', visible=False, width=140)
        [6] Column
            [0] Markdown(str)
            [1] Markdown(str)
            [2] MultiSelect(name='Selected Ranges')
            [3] Button(button_type='danger', name='Delete Selection')
    [1] HoloViews(DynamicMap)

## Save the labeled data
The cell below saves the data you labeled above, so that you may come back to it later on if needed. 

In [3]:
FF.save_frame_ranges()
print(f'Labeled flip classifier data saved to: {FF.training_data_path}')

Labeled flip classifier data saved to: /n/groups/datta/jlove/data/full_test_ds/full_test_ds/flip-training-frame-ranges.p


## Specify cleaning parameters
The cleaning parameters specified in your config file during extraction will be loaded below and will be used to generate a training dataset. If you wish to permute these parameters to test how that impacts your classifier's results PLEASE MAKE SURE TO TAKE NOTE OF THE PARAMETERS YOU CHANGE. There is not a functionality built in that allows you to automate this. 

In [4]:
# import cleaning params
config_data = read_yaml(config_path) # load config data
clean_parameters = {'prefilter_space': config_data['spatial_filter_size'], # median filter kernel sizes 
                    'prefilter_time': config_data['temporal_filter_size'], # temporal filter kernel sizes
                    'strel_tail': config_data['tail_filter_size'], # struct. element for filtering tail
                    'iters_tail': config_data['tail_filter_iters'], # number of iters for morph. opening to filter tail
                    'strel_min':config_data['cable_filter_size'], # structuring element for erosion
                    'iters_min': config_data['cable_filter_iters'], # number of iters for cable
                    'height_threshold': config_data['min_height']}# minimum height of mouse
clean_dataclass = CleanParameters(**clean_parameters)

## Train the classifier
The cell below will take the data you labeled, save the frames as a training dataset, and train a classifier on it. You can specify whether you want to use a support vector machine (`SVM`) or a random forest classifier (`RF`). You may read more about each here: [SVM](https://scikit-learn.org/stable/modules/svm.html), [RF](https://scikit-learn.org/stable/modules/ensemble.html). 

In [5]:
train_path = create_training_dataset(FF.training_data_path)
print(f'Flip classifier training data saved to: {train_path}')

clf = train_classifier(train_path, classifier='RF')
print('Flip classifier succesfully trained')

save_classifier(clf, model_path)

Training data shape: (72, 80, 80); memory usage: 0.00 GB
Flip classifier training data saved to: /n/groups/datta/jlove/data/full_test_ds/full_test_ds/training_data.npz
Fitting PCA
Running cross-validation
Held-out model accuracy: 0.9027777777777777
Final fitting step
Flip classifier succesfully trained
Classifier saved to /n/groups/datta/jlove/data/full_test_ds/full_test_ds/new-flip-tst.pkl


/home/jal5475/.miniconda/envs/moseq2-dev/lib/python3.7/site-packages/sklearn/utils/validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':
/home/jal5475/.miniconda/envs/moseq2-dev/lib/python3.7/site-packages/sklearn/utils/validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':
/home/jal5475/.miniconda/envs/moseq2-dev/lib/python3.7/site-packages/sklearn/utils/validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':
/home/jal5475/.miniconda/envs/moseq2-dev/lib/python3.7/site-packages/sklearn/utils/validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':
/home/jal5475/.miniconda/envs/moseq2-dev/lib

## Correct Extracted Dataset Using Train Flip Classifier Model

Use a pre-trained flip classifier model to correct extractions in your dataset that may have frames where the rodent is incorrectly flipped. 
### Apply a flip classifier to correct the extracted dataset
**Instructions:**
- **Set the `write_movie` variable to `True`** if you want to write a new video with the corrected frames.
- **Set the `verbose` variable to `True`** if you want to display progress bars for each session.
- **Run this cell** to apply the trained model to correct the extracted dataset.

In [6]:
chunk_size = 4000
frame_path = 'frames'
write_movie = True
verbose = False

FF.apply_flip_classifier(
    clf_path=model_path,
    chunk_size=chunk_size,
    frame_path=frame_path,
    write_movie=write_movie,
    verbose=verbose)

Flipping extracted sessions...:  15%|█▍        | 4/27 [03:53<22:22, 58.39s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jal5475/.miniconda/envs/moseq2-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-fa43950420dc>", line 11, in <module>
    verbose=verbose)
  File "/home/jal5475/moseq/source/moseq2-app/moseq2_app/flip/widget.py", line 236, in apply_flip_classifier
    progress_bar=verbose)
  File "/home/jal5475/moseq/source/moseq2-extract/moseq2_extract/io/video.py", line 454, in write_frames_preview
    disp_img = np.delete(use_cmap(disp_img), 3, 2)*255
  File "/home/jal5475/.miniconda/envs/moseq2-dev/lib/python3.7/site-packages/matplotlib/colors.py", line 536, in __call__
    xa[xa < 0] = self._i_under
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jal5475/.miniconda/envs/moseq2-dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py", li

KeyboardInterrupt: 

### Preview Corrected Sessions
**Instructions:**
- **Run the following cell** to preview corrected sessions.

In [ ]:
from moseq2_app.main import preview_extractions

preview_extractions(input_dir, flipped=True)

***